In [2]:
# Look for ones in the network that we're missing from our classification.
import os
import json

senders = {}
recipients = {}
inputdir = "./final/"
for subdir, dirs, files in os.walk(inputdir):
    for file in files:
        if file[-2:] != "js":
            continue
        json_file = open(subdir + os.sep + file)
        data = json.load(json_file)
        
        for k in ["transactions", "words"]:
            for x in data[k]:
                if "description" not in x:
                    continue
                if x["description"] == "sender":
                   senders[x["transactionID"]] = x["transliteratedWord"] 
                if x["description"] == "recipient":
                   recipients[x["transactionID"]] = x["transliteratedWord"] 
                

print(len(senders), len(recipients))
print(sorted([k for k in senders if k not in recipients]))
print(sorted([k for k in recipients if k not in senders]))

595 567
['ARKH3a-3', 'HT100-0', 'HT100-1', 'HT116a-1', 'HT117a-4', 'HT122b-1', 'HT125a-0', 'HT125b-0', 'HT131a-0', 'HT131b-0', 'HT23a-2', 'HT23a-3', 'HT23a-4', 'HT28a-1', 'HT28a-4', 'HT30-0', 'HT30-2', 'HT32-0', 'HT32-3', 'HT32-4', 'HT58-2', 'HT7a-1', 'HT88-4', 'HT94b-1', 'HT94b-7', 'HT95a-1', 'KH9-3', 'PE1-1']
[]


In [3]:
""" Get the top connected edges and display them in a table """

# Create a list of all edges
edges = [(senders[k], recipients[k]) for k in recipients]
edges += [(recipients[k], senders[k]) for k in recipients]
edges = sorted(set(edges))

# Create a table with the most connected nodes at the top.
from collections import defaultdict
node_count = defaultdict(int)
for f,t in edges:
    node_count[f] += 1
sorted_node_count = dict(sorted(node_count.items(), key=lambda x:x[1], reverse=True))

import pandas as pd
pd.set_option('display.max_rows', None)
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]

df = pd.DataFrame.from_dict(sorted_node_count, orient='index', columns=['Count'])
df.style.set_caption("Most connected Nodes").set_table_styles(styles)



,Count
Haghia Triada Magazine,126
Zakros Magazine,64
KA-PA,22
KI-RO,18
A-DU,17
PA,15
]-RA-RI,15
KI-KI-RA-JA,11
MA-KA-RI-TE,11
U-MI-NA-SI,11


In [63]:
import json
# Print out the top ten connected nodes
print(json.dumps([x for x,y in sorted(node_count.items(), key=lambda x:x[1], reverse=True)[:10]]))
                                                            

["Haghia Triada Magazine", "Zakros Magazine", "KA-PA", "KI-RO", "A-DU", "PA", "]-RA-RI", "KI-KI-RA-JA", "MA-KA-RI-TE", "U-MI-NA-SI"]


# Acyclic paths in the graph

In [6]:
# Find the longest acyclic paths up to a maximum of X.
def extendPath(p):
    legs = [p + [t] for f, t in edges if f == p[-1] and t not in p and "Magazine" not in t]
    return legs

def getMaxPathLength(n, maxLen=10):
    paths = [[n]]
    i = 0
    while i < maxLen:
        new_paths = []
        for p in paths:
            new_paths += extendPath(p)
        if not new_paths:
            break
        paths = new_paths
        i+=1
    return (i, len(paths), paths)


starting_nodes = list(set([f for f,t in edges]))
path_lengths = [(n,c) for n,c in [(n, getMaxPathLength(n,8)) for n in starting_nodes] if c[0] > 0]


In [7]:
df = pd.DataFrame([(a,b,c) for a,(b,c,d) in sorted(path_lengths, key=lambda p: p[1][1], reverse=True)],
                  columns=["Node", "Path Length", "No. of Paths"])
df.style.hide_index().set_caption("Most connected Nodes").set_table_styles(styles)


Node,Path Length,No. of Paths
Haghia Triada Magazine,8,117309
KU-RA-MU,8,43615
MI-TU,8,43615
TU-JU-MA,8,43615
MI-RU-TA-RA-RE,8,43615
MA-RU,8,43615
U-DI-MI,8,43615
TE-JA-RE,8,43615
U-SU,8,43615
NA-DA-RE,8,43615


In [8]:
# Get a list of nodes in the longest paths that all contain a given node
flatten = lambda l: [item for sublist in l for item in sublist]

l = list(set(flatten([p for a,(b,c,d) in path_lengths for p in d if "MA-RU" in p])))
l

['U-TI',
 'JA-*21F',
 'SA-PO',
 '*21F-TU-NE',
 'DA-ME',
 'Haghia Triada Magazine',
 'KI-RO',
 'KI-RE-TA₂',
 'TE-KI',
 'SA-RA₂',
 'QA-NU-MA',
 'DE-DI',
 'KA-U-DE-TA',
 'DI-DE-RU',
 'TU-MA',
 'PA',
 'U-SU',
 'QA-RA₂-WA',
 'I-RU-JA',
 'I-TA-JA',
 'Kharnia Magazine',
 'DI-DI-ZA-KE',
 'I-DU-NE-SI',
 'SA-RO',
 'QA-QA-RU',
 'A-DU',
 'SI-KI-NE',
 'MA-DI',
 'DU-DA-MA',
 'JA-MI-DA-RE',
 'RE-TE',
 'KA-SA-RU',
 ']-RA-RI',
 'QE-RA₂-U',
 'MA-*321',
 'DA-RU-*329',
 'A-SI',
 'TA',
 'MA-*79',
 '*188',
 'SA-RU',
 'KI-RE-TA-NA',
 'KU-NI-SU',
 'MA-RU',
 'Tylissos Magazine',
 'KU-PA₃-NU',
 'MA-KA-RI-TE',
 'SI-RU-MA-RI-TA₂',
 'KI-KI-RA-JA',
 'U-MI-NA-SI',
 'QA-*310-I',
 '*312-TA',
 'KA-PA',
 'DU',
 'SU-DU',
 'U-DI-MI',
 'PI-*34-TE',
 'QE-TI',
 'WI-DI-NA',
 'WI',
 'RE-DI-SE',
 'MI-NU-TE',
 'DA-RE',
 'PA-[I-TO',
 'TA-RI-NA',
 'I-KU-RI-NA',
 'RA-TI-SE',
 'WA-DU-NI-MI',
 'SA-MA',
 'DI',
 'U-TA-RO',
 'DA-TA-RA',
 'TA-NA-TI',
 'ME-ZA',
 'KU-RA-MU',
 'DA-DU-MA-TA',
 'PU-RA₂',
 'DI-NA-U',
 'DI-DE',
 'U-*325-ZA',
 '

# Cyclic Paths in the Graph

In [15]:
# Find the longest cyclic paths up to a maximum of X.
def getPaths(p):
    legs = [p + [t] for f, t in edges 
            if f == p[-1] and (t == p[0] or t not in p) and "Magazine" not in t]
    return legs

def getCycles(n, maxLen=10):
    paths = [[n]]
    cycles = []
    i = 0
    while i < maxLen:
        new_paths = []
        for p in paths:
            new_paths += getPaths(p)
        if not new_paths:
            break
        paths = [p for p in new_paths if p[0] != p[-1]]
        cycles += [p for p in new_paths if p[0] == p[-1]]
        i+=1
    if not len(cycles):
        return (0,0,[])
    return (max([len(c) for c in cycles]), len(cycles), sorted(cycles, key=lambda l: len(l), reverse=True))

# Store results in a dict while we try different path lengths.
path_lengths = {}


In [18]:
%%time

import multiprocessing

# split a list into evenly sized chunks
def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

MAX_PATH_LENGTH = 11
def worker(nodes):
    path_lengths = [(n,c) for n,c in [(n, getCycles(n, MAX_PATH_LENGTH)) for n in nodes] if c[0] > 0]
    return path_lengths

starting_nodes = list(set([f for f,t in edges]))
total = len(starting_nodes)
chunk_size = total / 10
batches = chunks(starting_nodes, int(chunk_size))

pool = multiprocessing.Pool(processes = 3)
path_lengths[MAX_PATH_LENGTH] = flatten(pool.map(worker, batches))


CPU times: user 913 ms, sys: 129 ms, total: 1.04 s
Wall time: 13min 14s


Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-21:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **se

In [22]:
df = pd.DataFrame([(a,b,c,d[0]) for a,(b,c,d) in sorted(path_lengths[MAX_PATH_LENGTH], key=lambda p: p[1][1], reverse=True)],
                  columns=["Node", "Max Path Length", "No. of Cyclic Paths", "Example"])
df.style.hide_index().set_caption("Most cyclic paths").set_table_styles(styles)

Node,Max Path Length,No. of Cyclic Paths,Example
KI-RO,12,21159,"['KI-RO', 'KU-PA₃-NU', 'MA-KA-RI-TE', '*21F-TU-NE', 'QE-TI', 'DA-RE', 'A-DU', 'SA-RA₂', 'A-SI-JA-KA', 'U-MI-NA-SI', 'KU-RA-MU', 'KI-RO']"
A-DU,12,20114,"['A-DU', 'DA-ME', 'DA-DU-MA-TA', 'DI-DE-RU', 'A-KA-RU', 'SA-RU', 'KI-RO', 'KU-PA₃-NU', 'U-MI-NA-SI', 'A-SI-JA-KA', 'SA-RA₂', 'A-DU']"
U-MI-NA-SI,12,17565,"['U-MI-NA-SI', 'A-SI-JA-KA', 'SA-RA₂', 'A-DU', 'DA-ME', 'DA-DU-MA-TA', 'DI-DE-RU', 'A-KA-RU', 'SA-RU', 'KI-RO', 'KU-PA₃-NU', 'U-MI-NA-SI']"
MA-KA-RI-TE,12,17343,"['MA-KA-RI-TE', '*21F-TU-NE', 'QE-TI', 'DA-RE', 'A-DU', 'DA-ME', 'DA-DU-MA-TA', 'QE-RA₂-U', 'KU-PA₃-NU', 'KI-RO', 'KU-RA-MU', 'MA-KA-RI-TE']"
KU-PA₃-NU,12,16127,"['KU-PA₃-NU', 'KI-RO', 'KU-RA-MU', 'MA-KA-RI-TE', '*21F-TU-NE', 'QE-TI', 'DA-RE', 'A-DU', 'DA-ME', 'DA-DU-MA-TA', 'QE-RA₂-U', 'KU-PA₃-NU']"
QE-RA₂-U,12,14169,"['QE-RA₂-U', 'A-DU', 'DA-ME', 'DA-DU-MA-TA', 'DI-DE-RU', 'A-KA-RU', 'SA-RU', 'KI-RO', 'KU-RA-MU', 'MA-KA-RI-TE', 'KU-PA₃-NU', 'QE-RA₂-U']"
SA-RU,12,11482,"['SA-RU', 'A-DU', 'DA-ME', 'DA-DU-MA-TA', 'QE-RA₂-U', 'KU-PA₃-NU', 'MA-KA-RI-TE', 'KU-RA-MU', 'U-MI-NA-SI', 'MA-RU', 'KI-RO', 'SA-RU']"
DA-DU-MA-TA,12,10532,"['DA-DU-MA-TA', 'DA-ME', 'A-DU', 'DA-RE', 'JE-DI', 'DI', 'KI-KI-RA-JA', 'QA-*310-I', ']-RA-RI', 'KU-PA₃-NU', 'QE-RA₂-U', 'DA-DU-MA-TA']"
]-RA-RI,12,9217,"[']-RA-RI', '*306-TU', 'SA-RO', 'TA', 'SI+SE', 'U-TA-RO', 'PU-RA₂', 'A-SI-JA-KA', 'SA-RA₂', 'A-DU', 'DA-RI-DA', ']-RA-RI']"
SA-RA₂,12,7126,"['SA-RA₂', 'A-DU', 'DA-ME', 'DA-DU-MA-TA', 'DI-DE-RU', 'A-KA-RU', 'SA-RU', 'KI-RO', 'KU-PA₃-NU', 'U-MI-NA-SI', 'A-SI-JA-KA', 'SA-RA₂']"


In [48]:
def isSameCycle(c1,c2):
    c1 = c1[:-1]
    c2 = c2[:-1]
    if len(c1) != len(c2):
        return False
    if sorted(c1) != sorted(c2):
        return False
    s = c2.index(c1[0])
    c2a = c2[s:] + c2[:s]
    if c1 == c2a:
        return True
    print(c1,c2a)
    return False

In [49]:
c1 = ['DA-DU-MA-TA', 'DA-ME', 'A-DU', 'DA-RE', 'JE-DI', 'DI', 'KI-KI-RA-JA', 'QA-*310-I', ']-RA-RI', 'KU-PA₃-NU', 'QE-RA₂-U', 'DA-DU-MA-TA']
c2 = ['DA-RE', 'JE-DI', 'DI', 'KI-KI-RA-JA', 'QA-*310-I', ']-RA-RI', 'KU-PA₃-NU', 'QE-RA₂-U', 'DA-DU-MA-TA', 'DA-ME', 'A-DU', 'DA-RE']

isSameCycle(c1,c2)


True

In [18]:
%%time

import networkx as nx
import matplotlib.pyplot as plt

# Create Directed Graph
G=nx.Graph()

# Create a list of all edges
edges = [(senders[k], recipients[k]) for k in recipients]
edges = [(recipients[k], senders[k]) for k in recipients]
edges = sorted(set(edges))
#edges = [(a,b) for (a,b) in edges if "Magazine" not in a and "Magazine" not in b]

nodes = sorted(set([a for (a,b) in edges] + [b for (a,b) in edges]))

# Add a list of nodes:
G.add_nodes_from(nodes)

# Add a list of edges:
G.add_edges_from(edges)



CPU times: user 3.3 ms, sys: 0 ns, total: 3.3 ms
Wall time: 3.21 ms


In [ ]:
#Return a list of cycles described as a list o nodes
cycles = list(nx.simple_cycles(G))
cycles


In [23]:
from networkx.algorithms.community import k_clique_communities
list(k_clique_communities(G, 3))

[frozenset({'*21F-*118', '*28B-NU-MA-RE', 'SI-PI-KI', 'Zakros Magazine'}),
 frozenset({'A-DU',
            'A-KA-RU',
            'DA-SI-*118',
            'Haghia Triada Magazine',
            'I-KA',
            'KA-PA',
            'KI-RI-TA₂',
            'KI-RO',
            'KU-PA₃-NU',
            'MI-NU-TE',
            'QE-RA₂-U',
            'SA-RA₂',
            'SA-RU'}),
 frozenset({'*306-TU',
            'DI-NA-U',
            'Haghia Triada Magazine',
            'SA-RO',
            'TA-I-AROM'}),
 frozenset({'A-RI-NI-TA', 'Haghia Triada Magazine', 'KI-RA'}),
 frozenset({'DI', 'Haghia Triada Magazine', 'JE-DI'}),
 frozenset({'KI-KI-RA-JA', 'ME-ZA', 'PA'})]

In [25]:
import networkx.algorithms.community as nx_comm
nx_comm.louvain_communities(G, seed=123)

[{'*301-NA', 'KU-PA-ZU', 'WI-NA-DU'},
 {'*301-U-RA',
  'A-SE-JA',
  'KA-PO-RU',
  'NA-*21F-NE-MI-NA',
  'PA-RA-NE',
  'SE-KU-TU'},
 {'*312-TE-TE', 'MI-KI-SE-NA', 'TA-TI', 'U-NU-*21F'},
 {'A-*325-ZA',
  'A-RI-PA',
  'Petras Magazine',
  'QA-QA-DA',
  'RU-PI-*305-MI',
  'TO-*49-RE',
  'TO-ME'},
 {'*21F-TU-NE',
  '*301',
  '*305-RU',
  '*306-KI-TA₂',
  '*312-TA',
  'A-RA-JU-U-DE-ZA',
  'DA-RE',
  'DA-RI-DA',
  'DA-RU-*329',
  'DI',
  'DU-JA',
  'I-RU-JA',
  'JE-DI',
  'KA',
  'KA-SA-RU',
  'KI-KI-RA-JA',
  'KI-RE-TA₂',
  'MA-DI',
  'ME-ZA',
  'PA',
  'QA-*310-I',
  'QA-QA-RU',
  'QE-KA',
  'QE-TI',
  'RE-DI-SE',
  'TA-NA-TI',
  'TA-RI-NA',
  'TE-TU',
  'U-TI',
  'WA-DU-NI-MI'},
 {'A-DA-KI-SI-KA', 'A-RA-U-DA', 'WI-SA-SA-NE'},
 {'A-DA-RO',
  'A-DU-NI-TA-NA',
  'Arkhalkhori Magazine',
  'JA-PI',
  'KA-NE',
  'KI-NU',
  'PI-*314'},
 {'A-DA',
  'A-DU',
  'A-KA-RU',
  'A-KU-TU-*361',
  'DA-DU-MA-TA',
  'DA-ME',
  'DA-QE-RA',
  'DA-U-*49',
  'DI-DE',
  'DI-DE-RU',
  'KI-RE-TA-NA',
  'KI-RI-SI',
